# Changes Suggestions
1. collaborative filtering 
2. user user and item item function (based on Frequent patterns- association rule mining)
3. bloom filter
4. time taken for user-user is too long--- Can utilize HDFS


In [37]:
pip install mlxtend
pip install pybloom-live

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 64.2 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.22.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.10.0, but you have google-cloud-bigquery 2.34.4 which is incompatible.
bigframes 0.22.0 requires google-cloud-storage>=2.0.0, but you have google-cloud-storage 1.44.0 which is incompatible.
bigframes 0.22.0 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.2.3 which is incompatible.
cesium 0.12.3 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
dataproc-jupyter-plugin 0.1.79 requires pydantic~=1.10.0, but you have pydantic 2.10.2 which is incompatible.
Note

In [39]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules
import hashlib
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/symptom-checker/Training.csv
/kaggle/input/symptom-checker/Testing.csv
/kaggle/input/disease-symptom-description-dataset/symptom_Description.csv
/kaggle/input/disease-symptom-description-dataset/Symptom-severity.csv
/kaggle/input/disease-symptom-description-dataset/symptom_precaution.csv
/kaggle/input/disease-symptom-description-dataset/dataset.csv


In [40]:
train_check = pd.read_csv('/kaggle/input/symptom-checker/Training.csv')
test_check = pd.read_csv('/kaggle/input/symptom-checker/Testing.csv')


In [41]:
dataset = pd.read_csv("/kaggle/input/disease-symptom-description-dataset/dataset.csv")

In [43]:
def clean_symptoms(df):
    """
    Clean symptom columns by removing NaN values and creating a list of symptoms
    """
    # Identify symptom columns
    symptom_cols = [col for col in df.columns if col.startswith('Symptom_')]
    
    # Create a new column with non-null symptoms as a list
    df['symptoms'] = df[symptom_cols].apply(lambda row: row.dropna().tolist(), axis=1)
    
    # Drop original symptom columns if needed
    df = df.drop(columns=symptom_cols)
    
    return df
    
cleaned_df = clean_symptoms(dataset)


In [55]:
cleaned_df

,Disease,symptoms
0,Fungal infection,"[itching, skin_rash, nodal_skin_eruptions, ..."
1,Fungal infection,"[ skin_rash, nodal_skin_eruptions, dischromi..."
2,Fungal infection,"[itching, nodal_skin_eruptions, dischromic _..."
3,Fungal infection,"[itching, skin_rash, dischromic _patches]"
4,Fungal infection,"[itching, skin_rash, nodal_skin_eruptions]"
...,...,...
4915,(vertigo) Paroymsal Positional Vertigo,"[ vomiting, headache, nausea, spinning_move..."
4916,Acne,"[ skin_rash, pus_filled_pimples, blackheads,..."
4917,Urinary tract infection,"[ burning_micturition, bladder_discomfort, f..."
4918,Psoriasis,"[ skin_rash, joint_pain, skin_peeling, silv..."


In [56]:
unique_items = set(item for transaction in cleaned_df["symptoms"] for item in transaction)
df = pd.DataFrame([{item: (item in transaction) for item in unique_items} for transaction in cleaned_df["symptoms"]])

In [57]:
df

,chills,nausea,blister,mucoid_sputum,weakness_in_limbs,watering_from_eyes,breathlessness,coma,hip_joint_pain,depression,...,painful_walking,swelling_of_stomach,acidity,passage_of_gases,polyuria,back_pain,enlarged_thyroid,history_of_alcohol_consumption,headache,diarrhoea
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4915,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
4916,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4917,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4918,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [69]:
frequent_itemsets = fpgrowth(df, min_support=0.05, use_colnames=True)
# Display results
print("Frequent Itemsets", frequent_itemsets)

# Generate association rules
as_rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.8, num_itemsets = 5)
print("Assosiation Rule", as_rules)



Frequent Itemsets       support                                           itemsets
0    0.159756                                       ( skin_rash)
1    0.137805                                          (itching)
2    0.162195                                          ( chills)
3    0.389024                                        ( vomiting)
4    0.141463                                      ( chest_pain)
..        ...                                                ...
557  0.057317  ( chills,  high_fever,  fatigue,  phlegm,  cough)
558  0.058537  ( high_fever,  fatigue,  chest_pain,  phlegm, ...
559  0.057317  ( chills,  high_fever,  fatigue,  chest_pain, ...
560  0.054878  ( chills,  high_fever,  fatigue,  phlegm,  cou...
561  0.054878  ( chills,  high_fever,  fatigue,  chest_pain, ...

[562 rows x 2 columns]
Assosiation Rule                          antecedents  \
0         ( abdominal_pain, itching)   
1         ( yellowish_skin, itching)   
2                          ( chills)   
3

In [72]:
def rule_suggestion(initial_symptom, association_rules):
    updated_symptoms = list(initial_symptom)
    rules = association_rules.toPandas()
    for i in range(len(rules)):
        antecedent = rules['antecedent'][i]
        consequent = rules['consequent'][i]
        if set(map(str.strip, antecedent)) == set(init_symp):
            updated_symptoms.append(consequent[0])
            break
    return updated_symptoms


In [77]:
rule_suggestion(init_symp, as_rules)

In [78]:
disease_list = dataset['Disease'].unique()

In [79]:
# Iterate over each disease
disease_symptom_vocab = {}
for disease in disease_list:
    unique_symptoms = set()  # Use a set to avoid duplicate symptoms
    
    # Loop through all symptom columns
    for i in range(17):
        column_name = f'Symptom_{i+1}'
        # Extract unique symptoms for the current disease and column
        symptoms = pd.unique(dataset.loc[dataset["Disease"] == disease, column_name])
        # Add symptoms to the set, ignoring NaN values
        unique_symptoms.update([symptom.strip() for symptom in symptoms if pd.notna(symptom)])
    
    # Add to the disease vocabulary
    disease_symptom_vocab[disease] = list(unique_symptoms)

# Print the vocabulary for each disease
for disease, symptoms in disease_symptom_vocab.items():
    print("=============", f"Disease: {disease}", "================")
    print(f"Unique Symptoms: {symptoms}")

# Create a master vocabulary (all unique symptoms across diseases)
master_symptom_vocab = set()
for symptoms in disease_symptom_vocab.values():
    master_symptom_vocab.update(symptoms)

print("\nMaster Symptom Vocabulary:")
print(sorted(master_symptom_vocab))  # Sorted for better readability

============= Disease: Fungal infection ================
Unique Symptoms: ['nodal_skin_eruptions', 'skin_rash', 'itching', 'dischromic _patches']
============= Disease: Allergy ================
Unique Symptoms: ['continuous_sneezing', 'chills', 'shivering', 'watering_from_eyes']
============= Disease: GERD ================
Unique Symptoms: ['acidity', 'cough', 'ulcers_on_tongue', 'vomiting', 'stomach_pain', 'chest_pain']
============= Disease: Chronic cholestasis ================
Unique Symptoms: ['itching', 'abdominal_pain', 'yellowing_of_eyes', 'yellowish_skin', 'nausea', 'loss_of_appetite', 'vomiting']
============= Disease: Drug Reaction ================
Unique Symptoms: ['skin_rash', 'itching', 'burning_micturition', 'stomach_pain', 'spotting_ urination']
============= Disease: Peptic ulcer diseae ================
Unique Symptoms: ['internal_itching', 'abdominal_pain', 'indigestion', 'loss_of_appetite', 'vomiting', 'passage_of_gases']
============= Disease: AIDS ================
U

In [80]:
# Create a mapping of symptoms to unique indices
master_symptom_vocab_idx = {symptom: idx for idx, symptom in enumerate(sorted(master_symptom_vocab))}

# Print the indexed vocabulary
print("Indexed Symptom Vocabulary:")
for symptom, idx in master_symptom_vocab_idx.items():
    print(f"{idx}: {symptom}")

Indexed Symptom Vocabulary:
0: abdominal_pain
1: abnormal_menstruation
2: acidity
3: acute_liver_failure
4: altered_sensorium
5: anxiety
6: back_pain
7: belly_pain
8: blackheads
9: bladder_discomfort
10: blister
11: blood_in_sputum
12: bloody_stool
13: blurred_and_distorted_vision
14: breathlessness
15: brittle_nails
16: bruising
17: burning_micturition
18: chest_pain
19: chills
20: cold_hands_and_feets
21: coma
22: congestion
23: constipation
24: continuous_feel_of_urine
25: continuous_sneezing
26: cough
27: cramps
28: dark_urine
29: dehydration
30: depression
31: diarrhoea
32: dischromic _patches
33: distention_of_abdomen
34: dizziness
35: drying_and_tingling_lips
36: enlarged_thyroid
37: excessive_hunger
38: extra_marital_contacts
39: family_history
40: fast_heart_rate
41: fatigue
42: fluid_overload
43: foul_smell_of urine
44: headache
45: high_fever
46: hip_joint_pain
47: history_of_alcohol_consumption
48: increased_appetite
49: indigestion
50: inflammatory_nails
51: internal_itchi

In [81]:
import numpy as np

# Initialize a dictionary to store binary vectors for each disease
disease_vectors = {}

# Get a list of unique diseases
disease_list = dataset['Disease'].unique()

# Iterate through each disease
for disease in disease_list:
    # Initialize a binary vector of length equal to the master symptom vocabulary
    binary_vector = np.zeros(len(master_symptom_vocab_idx), dtype=int)
    
    # Iterate through all symptom columns for the current disease
    for i in range(17):
        column_name = f'Symptom_{i+1}'
        # Extract symptoms for the current disease and column
        symptoms = dataset.loc[dataset['Disease'] == disease, column_name]
        # Mark the corresponding index in the binary vector as 1 for each symptom
        for symptom in symptoms:
            if pd.notna(symptom):  # Ignore NaN values
                symptom = symptom.strip()  # Clean symptom string
                if symptom in master_symptom_vocab_idx:  # Ensure symptom exists in the vocabulary
                    binary_vector[master_symptom_vocab_idx[symptom]] = 1
    
    # Store the binary vector for the disease
    disease_vectors[disease] = binary_vector

# Convert the dictionary into a DataFrame for better readability
disease_vectors_df = pd.DataFrame.from_dict(
    disease_vectors, orient='index', columns=sorted(master_symptom_vocab_idx)
)


In [17]:
disease_vectors_df

,abdominal pain,abnormal menstruation,acidity,acute liver failure,altered sensorium,anxiety,back pain,belly pain,blackheads,bladder discomfort,...,vomiting,watering from eyes,weakness in limbs,weakness of one body side,weight gain,weight loss,yellow crust ooze,yellow urine,yellowing of eyes,yellowish skin
Fungal infection,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Allergy,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
GERD,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
Chronic cholestasis,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,1
Drug Reaction,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Peptic ulcer diseae,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
AIDS,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Diabetes,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
Gastroenteritis,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
Bronchial Asthma,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
from sklearn.metrics import jaccard_score


In [ ]:
train_check.head()

In [96]:
def reorder_columns(dataset, bitvector, correct_order):
    # Reorder dataset columns
    dataset = dataset[correct_order]
    # Reorder bitvector
    reordered_bitvector = [bitvector[correct_order.index(col)] for col in dataset.columns]
    return dataset, reordered_bitvector


In [97]:
# new_train = train_check[master_symptom_vocab]
new_train = sorted(train_check.columns)

users_profiles = train_check[new_train]
train_disease = users_profiles.pop('prognosis')
users_profiles.drop('fluid_overload.1', axis=1, inplace=True)


/tmp/ipykernel_24/483330719.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users_profiles.drop('fluid_overload.1', axis=1, inplace=True)


In [99]:
our_new_patient= users_profiles.head(1)

In [100]:
users_profiles

,abdominal_pain,abnormal_menstruation,acidity,acute_liver_failure,altered_sensorium,anxiety,back_pain,belly_pain,blackheads,bladder_discomfort,...,vomiting,watering_from_eyes,weakness_in_limbs,weakness_of_one_body_side,weight_gain,weight_loss,yellow_crust_ooze,yellow_urine,yellowing_of_eyes,yellowish_skin
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4915,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4916,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4917,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4918,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
len(train_check)

20

In [36]:
from sklearn.metrics import jaccard_score

# Initialize an empty list to store the results
similarities = []


# Loop through all patients
for j in range(len(our_new_patient)):
    similiar_patients_index = []
    # Loop through all diseases
    for i in range(len(users_profiles)):
        rec_disease = {}
        
        # Compute the Jaccard similarity for disease `i` with patient `j`
        jaccard_sim = jaccard_score(users_profiles.iloc[i], our_new_patient.iloc[j])  # Compare one disease vs one patient
        rec_disease['similiar_patient'] = users_profiles.index[i]  # Store the disease name (index of the disease)
        rec_disease['new_patient'] = users_profiles.index[j]  # Store the patient name (index of the patient)
        rec_disease['jaccard_similarity'] = jaccard_sim  # Store the similarity score
        rec_disease["user_profiles_idx"] = i
        similarities.append(rec_disease)  # Add to the list
        
        
# Convert the list of similarities to a DataFrame for easier manipulation
similarities_df = pd.DataFrame(similarities)




             patient           disease  jaccard_similarity
patient                                                   
0       0          0  Fungal infection            1.000000
        4          0     Drug Reaction            0.285714
        16         0       Chicken pox            0.153846
        37         0              Acne            0.142857
        40         0          Impetigo            0.125000
...              ...               ...                 ...
19      780       19           Allergy            1.000000
        805       19       Common Cold            0.105263
        794       19           Malaria            0.090909
        797       19           Typhoid            0.071429
        806       19         Pneumonia            0.071429

[100 rows x 3 columns]


/tmp/ipykernel_24/1735050759.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_5_similar_diseases = similarities_df.groupby('patient').apply(lambda x: x.sort_values(by='jaccard_similarity', ascending=False).head(5))


In [ ]:
# Group by patient and sort by Jaccard similarity, then select top 5
similar_patients = similarities_df.groupby('new_patient').apply(lambda x: x.sort_values(by='jaccard_similarity', ascending=False)
similiar_patients["disease"] = train_disease[similar_patients["user_profiles_idx"]]
                                                                


# Bloom Filter

In [ ]:
from pybloom_live import BloomFilter

# Example size and false positive rate for the Bloom Filter
bloom_filter = BloomFilter(capacity=10000, error_rate=0.01)

# Iterate over the "disease" column in the dataframe
for disease in similiar_patients["disease"]:
    if disease in bloom_filter:
        # If the disease is already in the Bloom filter, do nothing
        pass
    else:
        # Add the disease to the Bloom filter and print it
        bloom_filter.add(disease)
        print("Disease:", disease)
